In [ ]:
pip install transformers datasets scikit-learn torch evaluate

In [4]:
import os

def load_annotations(annotations_file):
    annotations = []
    position = []
    protagonist_annotations = []
    antagonist_annotations = []
    innocent_annotations = []
    protagonist_positions = []
    antagonist_positions = []
    innocent_positions = []

    with open(annotations_file, 'r') as file:
        for line in file:
            parts = line.strip().split('\t')
            pos = [parts[2], parts[3]]
            position.append(pos)

            # Find the index where "Protagonist", "Antagonist", or "Innocent" appears
            for i, part in enumerate(parts):
                if part in ["Protagonist", "Antagonist", "Innocent"]:
                    main_info = parts[:i+1]  # Everything up to and including the found role
                    subcategories = parts[i+1:]  # Everything after the role
                    annotation_entry = main_info + [subcategories]
                    annotations.append(annotation_entry)

                    # Add to respective category list
                    if part == "Protagonist":
                        protagonist_annotations.append(annotation_entry)
                        protagonist_positions.append(pos)
                    elif part == "Antagonist":
                        antagonist_annotations.append(annotation_entry)
                        antagonist_positions.append(pos)
                    elif part == "Innocent":
                        innocent_annotations.append(annotation_entry)
                        innocent_positions.append(pos)

                    break

    return annotations, position, protagonist_annotations, antagonist_annotations, innocent_annotations, protagonist_positions, antagonist_positions, innocent_positions

def load_documents(raw_documents_folder):
    documents = {}
    for filename in os.listdir(raw_documents_folder):
        file_path = os.path.join(raw_documents_folder, filename)
        with open(file_path, 'r', encoding='utf-8') as file:
            documents[filename] = file.read()
    return documents

def process_documents(annotations_file, raw_documents_folder):
    annotations, position_array, protagonist_annotations, antagonist_annotations, innocent_annotations, protagonist_positions, antagonist_positions, innocent_positions = load_annotations(annotations_file)
    documents = load_documents(raw_documents_folder)

    result_protagonist = []
    result_antagonist = []
    result_innocent = []

    for annotation in protagonist_annotations:
        filename = annotation[0]
        if filename in documents:
            document_text = documents[filename]
            result_protagonist.append([document_text] + annotation)

    for annotation in antagonist_annotations:
        filename = annotation[0]
        if filename in documents:
            document_text = documents[filename]
            result_antagonist.append([document_text] + annotation)

    for annotation in innocent_annotations:
        filename = annotation[0]
        if filename in documents:
            document_text = documents[filename]
            result_innocent.append([document_text] + annotation)

    return result_protagonist, result_antagonist, result_innocent, protagonist_positions, antagonist_positions, innocent_positions

annotations_file = 'subtask-1-annotations.txt'
raw_documents_folder = 'raw-documents'
result_protagonist, result_antagonist, result_innocent, protagonist_positions, antagonist_positions, innocent_positions = process_documents(annotations_file, raw_documents_folder)


In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
import os
from datasets import DatasetDict, Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
import numpy as np
import torch

os.environ["WANDB_DISABLED"] = "true"

# Define label mappings for each category
protagonist_labels = {"Guardian": 0, "Martyr": 1, "Peacemaker": 2, "Rebel": 3, "Underdog": 4, "Virtuous": 5}
antagonist_labels = {"Instigator": 0, "Conspirator": 1, "Tyrant": 2, "Foreign Adversary": 3, "Traitor": 4, "Spy": 5,
                     "Saboteur": 6, "Corrupt": 7, "Incompetent": 8, "Terrorist": 9, "Deceiver": 10, "Bigot": 11}
innocent_labels = {"Forgotten": 0, "Exploited": 1, "Victim": 2, "Scapegoat": 3}

# Function to map labels to integers (handling multiple labels)
def map_labels(label_list, label_mapping):
    return [label_mapping[label] for label in label_list if label in label_mapping]

# Processed data for each category
protagonist_processed_data = [
    {"text": entry[0], "label": map_labels(entry[-1], protagonist_labels)}
    for entry in result_protagonist
]

antagonist_processed_data = [
    {"text": entry[0], "label": map_labels(entry[-1], antagonist_labels)}
    for entry in result_antagonist
]

innocent_processed_data = [
    {"text": entry[0], "label": map_labels(entry[-1], innocent_labels)}
    for entry in result_innocent
]

# Print sample processed data to verify
print("Protagonist Example:", protagonist_processed_data[:2])
print("Antagonist Example:", antagonist_processed_data[:2])
print("Innocent Example:", innocent_processed_data[:2])


In [ ]:
#Protagonist
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from torch.utils.data import Dataset, DataLoader
import torch
import evaluate
from transformers import AutoTokenizer
import torch.nn.functional as F

# Initialize tokenizer
tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-cased")

special_tokens = {"additional_special_tokens": ["[TARGET]", "[/TARGET]"]}
tokenizer.add_special_tokens(special_tokens)

# Update model to use multi-label classification (sigmoid activation instead of softmax)
num_labels = 12  # Adjust according to the total number of subcategories
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=num_labels, problem_type="multi_label_classification")
model.resize_token_embeddings(len(tokenizer))

def mark_target_word(text, position_range, start_token="[TARGET]", end_token="[/TARGET]", max_tokens=512):
    start_char, end_char = position_range
    marked_text = text[:int(start_char)] + start_token + text[int(start_char):int(end_char)] + end_token + text[int(end_char):]

    tokens = tokenizer.tokenize(marked_text)

    if len(tokens) > max_tokens:
        target_start_index = len(tokenizer.tokenize(text[:int(start_char)]))
        target_end_index = target_start_index + len(tokenizer.tokenize(text[int(start_char):int(end_char)]))

        context_size = (max_tokens - (target_end_index - target_start_index)) // 2

        start_index = max(0, target_start_index - context_size)
        end_index = min(len(tokens), target_end_index + context_size)

        tokens = tokens[start_index:end_index]
        marked_text = tokenizer.convert_tokens_to_string(tokens)

    return marked_text

def tokenize_function(text):
    return tokenizer(text, padding="max_length", truncation=True, max_length=128)

tokenized_data = []

for x in range(len(protagonist_processed_data)):
    text = protagonist_processed_data[x]['text']
    labels = protagonist_processed_data[x]['label']  # Expecting a list of labels
    position_range = protagonist_positions[x]

    marked_text = mark_target_word(text, position_range)
    tokenized_text = tokenize_function(marked_text)

    # Convert labels to a multi-hot encoding format
    multi_hot_labels = torch.zeros(num_labels, dtype=torch.float)
    for label in labels:
        multi_hot_labels[label] = 1.0

    tokenized_data.append({"input_ids": tokenized_text["input_ids"], "attention_mask": tokenized_text["attention_mask"], "labels": multi_hot_labels})

class TokenizedDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return {
            "input_ids": torch.tensor(self.data[idx]["input_ids"], dtype=torch.long),
            "attention_mask": torch.tensor(self.data[idx]["attention_mask"], dtype=torch.long),
            "labels": self.data[idx]["labels"]
        }

data_train, data_val = train_test_split(tokenized_data, test_size=0.2, random_state=42)

train_dataset = TokenizedDataset(data_train)
val_dataset = TokenizedDataset(data_val)

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    load_best_model_at_end=True
)

clf_metrics = evaluate.combine(["accuracy", "f1", "precision", "recall"])

def sigmoid(x):
   return 1/(1 + np.exp(-x))

def compute_metrics(eval_pred):

   predictions, labels = eval_pred
   predictions = sigmoid(predictions)
   predictions = (predictions > 0.5).astype(int).reshape(-1)
   return clf_metrics.compute(predictions=predictions, references=labels.astype(int).reshape(-1))

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

trainer.train()

results = trainer.evaluate()
print(f"F1 Score: {results['eval_f1']:.4f}")

model.save_pretrained("./text_classification_model")
tokenizer.save_pretrained("./text_classification_model")


In [ ]:
from huggingface_hub import HfApi, HfFolder
from transformers import AutoTokenizer, AutoModelForSequenceClassification

repo_name = "ProtagonistClassificationModel"

model.save_pretrained("./text_classification_model")
tokenizer.save_pretrained("./text_classification_model")

model.push_to_hub(repo_name)
tokenizer.push_to_hub(repo_name)

print(f"Model successfully pushed to: https://huggingface.co/{repo_name}")


In [ ]:
#Antagonist
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from torch.utils.data import Dataset, DataLoader
import torch
import evaluate
from transformers import AutoTokenizer
import torch.nn.functional as F

# Initialize tokenizer
tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-cased")

special_tokens = {"additional_special_tokens": ["[TARGET]", "[/TARGET]"]}
tokenizer.add_special_tokens(special_tokens)

num_labels = 12  # Adjust according to the total number of subcategories
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=num_labels, problem_type="multi_label_classification")
model.resize_token_embeddings(len(tokenizer))

def mark_target_word(text, position_range, start_token="[TARGET]", end_token="[/TARGET]", max_tokens=512):
    start_char, end_char = position_range
    marked_text = text[:int(start_char)] + start_token + text[int(start_char):int(end_char)] + end_token + text[int(end_char):]

    tokens = tokenizer.tokenize(marked_text)

    if len(tokens) > max_tokens:
        target_start_index = len(tokenizer.tokenize(text[:int(start_char)]))
        target_end_index = target_start_index + len(tokenizer.tokenize(text[int(start_char):int(end_char)]))

        context_size = (max_tokens - (target_end_index - target_start_index)) // 2

        start_index = max(0, target_start_index - context_size)
        end_index = min(len(tokens), target_end_index + context_size)

        tokens = tokens[start_index:end_index]
        marked_text = tokenizer.convert_tokens_to_string(tokens)

    return marked_text

def tokenize_function(text):
    return tokenizer(text, padding="max_length", truncation=True, max_length=128)

tokenized_data = []

for x in range(len(antagonist_processed_data)):
    text = antagonist_processed_data[x]['text']
    labels = antagonist_processed_data[x]['label']  # Expecting a list of labels
    position_range = antagonist_positions[x]

    marked_text = mark_target_word(text, position_range)
    tokenized_text = tokenize_function(marked_text)

    # Convert labels to a multi-hot encoding format
    multi_hot_labels = torch.zeros(num_labels, dtype=torch.float)
    for label in labels:
        multi_hot_labels[label] = 1.0

    tokenized_data.append({"input_ids": tokenized_text["input_ids"], "attention_mask": tokenized_text["attention_mask"], "labels": multi_hot_labels})

class TokenizedDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return {
            "input_ids": torch.tensor(self.data[idx]["input_ids"], dtype=torch.long),
            "attention_mask": torch.tensor(self.data[idx]["attention_mask"], dtype=torch.long),
            "labels": self.data[idx]["labels"]
        }

data_train, data_val = train_test_split(tokenized_data, test_size=0.2, random_state=42)

train_dataset = TokenizedDataset(data_train)
val_dataset = TokenizedDataset(data_val)

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    load_best_model_at_end=True
)

clf_metrics = evaluate.combine(["accuracy", "f1", "precision", "recall"])

def sigmoid(x):
   return 1/(1 + np.exp(-x))

def compute_metrics(eval_pred):
   print(eval_pred)
   predictions, labels = eval_pred
   predictions = sigmoid(predictions)
   predictions = (predictions > 0.5).astype(int).reshape(-1)
   return clf_metrics.compute(predictions=predictions, references=labels.astype(int).reshape(-1))

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

trainer.train()

results = trainer.evaluate()
print(f"F1 Score: {results['eval_f1']:.4f}")

model.save_pretrained("./text_classification_model")
tokenizer.save_pretrained("./text_classification_model")


In [ ]:
from huggingface_hub import HfApi, HfFolder
from transformers import AutoTokenizer, AutoModelForSequenceClassification

repo_name = "AntagonistClassificationModel"

model.save_pretrained("./text_classification_model")
tokenizer.save_pretrained("./text_classification_model")

model.push_to_hub(repo_name)
tokenizer.push_to_hub(repo_name)

print(f"Model successfully pushed to: https://huggingface.co/{repo_name}")


In [ ]:
#Innocent
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from torch.utils.data import Dataset, DataLoader
import torch
import evaluate
from transformers import AutoTokenizer
import torch.nn.functional as F

# Initialize tokenizer
tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-cased")

special_tokens = {"additional_special_tokens": ["[TARGET]", "[/TARGET]"]}
tokenizer.add_special_tokens(special_tokens)

# Update model to use multi-label classification (sigmoid activation instead of softmax)
num_labels = 4  # Adjust according to the total number of subcategories
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=num_labels, problem_type="multi_label_classification")
model.resize_token_embeddings(len(tokenizer))

def mark_target_word(text, position_range, start_token="[TARGET]", end_token="[/TARGET]", max_tokens=512):
    start_char, end_char = position_range
    marked_text = text[:int(start_char)] + start_token + text[int(start_char):int(end_char)] + end_token + text[int(end_char):]

    tokens = tokenizer.tokenize(marked_text)

    if len(tokens) > max_tokens:
        target_start_index = len(tokenizer.tokenize(text[:int(start_char)]))
        target_end_index = target_start_index + len(tokenizer.tokenize(text[int(start_char):int(end_char)]))

        context_size = (max_tokens - (target_end_index - target_start_index)) // 2

        start_index = max(0, target_start_index - context_size)
        end_index = min(len(tokens), target_end_index + context_size)

        tokens = tokens[start_index:end_index]
        marked_text = tokenizer.convert_tokens_to_string(tokens)

    return marked_text

def tokenize_function(text):
    return tokenizer(text, padding="max_length", truncation=True, max_length=128)

tokenized_data = []

for x in range(len(innocent_processed_data)):
    text = innocent_processed_data[x]['text']
    labels = innocent_processed_data[x]['label']  # Expecting a list of labels
    position_range = innocent_positions[x]

    marked_text = mark_target_word(text, position_range)
    tokenized_text = tokenize_function(marked_text)

    # Convert labels to a multi-hot encoding format
    multi_hot_labels = torch.zeros(num_labels, dtype=torch.float)
    for label in labels:
        multi_hot_labels[label] = 1.0

    tokenized_data.append({"input_ids": tokenized_text["input_ids"], "attention_mask": tokenized_text["attention_mask"], "labels": multi_hot_labels})

class TokenizedDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return {
            "input_ids": torch.tensor(self.data[idx]["input_ids"], dtype=torch.long),
            "attention_mask": torch.tensor(self.data[idx]["attention_mask"], dtype=torch.long),
            "labels": self.data[idx]["labels"]
        }

data_train, data_val = train_test_split(tokenized_data, test_size=0.2, random_state=42)

train_dataset = TokenizedDataset(data_train)
val_dataset = TokenizedDataset(data_val)

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    load_best_model_at_end=True
)

clf_metrics = evaluate.combine(["accuracy", "f1", "precision", "recall"])

def sigmoid(x):
   return 1/(1 + np.exp(-x))

def compute_metrics(eval_pred):

   predictions, labels = eval_pred
   predictions = sigmoid(predictions)
   predictions = (predictions > 0.5).astype(int).reshape(-1)
   return clf_metrics.compute(predictions=predictions, references=labels.astype(int).reshape(-1))

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

trainer.train()

results = trainer.evaluate()
print(f"F1 Score: {results['eval_f1']:.4f}")

model.save_pretrained("./text_classification_model")
tokenizer.save_pretrained("./text_classification_model")


In [ ]:
from huggingface_hub import HfApi, HfFolder
from transformers import AutoTokenizer, AutoModelForSequenceClassification

repo_name = "InnocentClassificationModel"

model.save_pretrained("./text_classification_model")
tokenizer.save_pretrained("./text_classification_model")

model.push_to_hub(repo_name)
tokenizer.push_to_hub(repo_name)

print(f"Model successfully pushed to: https://huggingface.co/{repo_name}")
